In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

ImportError: cannot import name 'r2_score' from 'sklearn.linear_model' (c:\Users\Hasnain Memon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\__init__.py)

In [50]:
df= pd.read_csv('homes_data.csv')
df.head()

,Price,Beds,Baths,Area,Location score
0,24 Crore,6.0,6.0,500 Sq. Yd.,5
1,25 Crore,6.0,7.0,500 Sq. Yd.,5
2,55 Crore,7.0,7.0,"1,000 Sq. Yd.",5
3,42 Crore,6.0,7.0,666 Sq. Yd.,5
4,21.75 Crore,6.0,6.0,500 Sq. Yd.,5


In [51]:
def convert_price(price):
    price= price.lower().replace(",", "").strip()
    
    if "crore" in price:
        return float(price.replace("crore", "").strip())
    elif "lakh" in price:
        return float(price.replace("lakh", "").strip())/100
    elif "arab" in price:
        return float(price.replace("arab", "").strip()) * 100
    else:
        return None
    

In [52]:
df["Price"]= df["Price"].apply(convert_price)
df["Price"]

0      24.00
1      25.00
2      55.00
3      42.00
4      21.75
       ...  
220     2.15
221     1.32
222     1.80
223     2.65
224     2.10
Name: Price, Length: 225, dtype: float64

In [53]:
df.head()

,Price,Beds,Baths,Area,Location score
0,24.00,6.0,6.0,500 Sq. Yd.,5
1,25.00,6.0,7.0,500 Sq. Yd.,5
2,55.00,7.0,7.0,"1,000 Sq. Yd.",5
3,42.00,6.0,7.0,666 Sq. Yd.,5
4,21.75,6.0,6.0,500 Sq. Yd.,5


In [54]:
def convert_area(area):
    area= area.lower().replace(",", "").replace("sq. yd.", "").strip()
    return float(area)


In [55]:
df["Area"]= df["Area"].apply(convert_area)
df.head()



,Price,Beds,Baths,Area,Location score
0,24.00,6.0,6.0,500.0,5
1,25.00,6.0,7.0,500.0,5
2,55.00,7.0,7.0,1000.0,5
3,42.00,6.0,7.0,666.0,5
4,21.75,6.0,6.0,500.0,5


In [56]:
df = df.dropna(subset=["Beds", "Baths"])


In [58]:
df = df[(df["Price"] <= 60) & (df["Area"] <= 1500)]


In [99]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x= df[["Beds", "Baths", "Area", "Location score"]]
y= df["Price"]

num_features= ["Beds", "Baths", "Area", "Location score"]
num_pipeline= Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor= ColumnTransformer(transformers=[
    ("num", num_pipeline, num_features)
    ]
)

model_pipeline= Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])




In [101]:
from sklearn.metrics import r2_score
scores=[]
for i in range(1,1000):
    x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=i)
    lr= LinearRegression()
    lr.fit(x_train,y_train)
    y_pred= lr.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

    

In [105]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=154)
model_pipeline.fit(x_train,y_train)
y_pred= model_pipeline.predict(x_test)
print(r2_score(y_test,y_pred))


0.9170627181923552


In [106]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")

MAE: 1.5580256153651142, MSE: 5.513237951865094, RMSE: 2.348028524499882


In [111]:
user_input = pd.DataFrame({
    "Beds": [6],
    "Baths": [6],
    "Area": [500],
    "Location score": [5]
})

predicted_price = model_pipeline.predict(user_input)
print("Predicted Price (Crore):", predicted_price[0])


Predicted Price (Crore): 16.53995959867582


In [110]:
import pickle
pickle.dump(model_pipeline,open('House_Prediction_Model.pkl','wb'))

In [115]:
from sklearn.preprocessing import MinMaxScaler

df["price_per_sq_yard"] = df["Price"] / df["Area"]

location_scores = (
    df.groupby("Location score")["price_per_sq_yard"]
    .mean()
    .reset_index()
)

scaler = MinMaxScaler(feature_range=(1, 10))
location_scores["Location score"] = scaler.fit_transform(
    location_scores[["price_per_sq_yard"]]
)

location_scores_dict = dict(
    zip(location_scores["Location score"], location_scores["Location score"])
)
pickle.dump(location_scores_dict, open("location_scores.pkl", "wb"))